In [1]:
import os
import re
import csv
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
from scipy.stats import gaussian_kde
import pandas as pd

In [6]:
os.chdir("E:/Temp")

In [7]:
liste_temperature = os.listdir()

In [19]:
def grep(liste_s,pattern):
    list_return=[]
    for s in (liste_s):
        res = ('\n'.join(re.findall(r'^.*%s.*?$'%pattern,s,flags=re.M)))
        if res != '':
            list_return.append(res)            
        
    return list_return

def is_composed(string,pattern):
    res = ('\n'.join(re.findall(r'^.*%s.*?$'%pattern,string,flags=re.M)))
    if res != '':
        return True
    else: 
        return False

In [10]:
cpt_102=0
# Creation d'un dataframe qui contient tous les resultats pour un champ
res_champ_102 = pd.DataFrame(columns=["temperature",\
                                   "numero_champ",\
                                   "nb_cellules",\
                                   "nb_cellules_mitose",\
                                   "%_mitose",\
                                   "nb_cellule_lagging",\
                                   "%_lagging",\
                                   "moyenne_duree_metaphase",\
                                   "std_duree_metaphase",\
                                   "moyenne_vitesse_metaphase",\
                                   "std_vitesse_metaphase",\
                                   "moyenne_duree_anaphase",\
                                   "std_duree_anaphase",\
                                   "moyenne_vitesse_anaphase",\
                                   "std_vitesse_anaphase"])

cpt_849=0
# Creation d'un dataframe qui contient tous les resultats pour un champ
res_champ_849 = pd.DataFrame(columns=["temperature",\
                                   "numero_champ",\
                                   "nb_cellules",\
                                   "nb_cellules_mitose",\
                                   "%_mitose",\
                                   "nb_cellule_lagging",\
                                   "%_lagging",\
                                   "moyenne_duree_metaphase",\
                                   "std_duree_metaphase",\
                                   "moyenne_vitesse_metaphase",\
                                   "std_vitesse_metaphase",\
                                   "moyenne_duree_anaphase",\
                                   "std_duree_anaphase",\
                                   "moyenne_vitesse_anaphase",\
                                   "std_vitesse_anaphase"])

for temperature in (liste_temperature) :
  
    liste_lignee = os.listdir(temperature)
    print(liste_lignee)
    for lignee in (liste_lignee) :
        liste_champ = os.listdir(temperature+"/"+lignee)
        print(liste_champ)
        for champ in (liste_champ) :          
            
            BF_Results = pd.read_csv(temperature+"/"+lignee+"/"+champ+"/BF_1/BF_Results.csv")
            #with open(temperature+"/"+lignee+"/"+champ+"/X0_Y0/BF_Results.csv","r") as file :
            #BF_Results = csv.DictReader(file, delimiter=',')
            list_intensite = list()
            list_intensite = BF_Results["Mean"]
            #print(intensite)
            #plt.hist(intensite,50, normed=1, facecolor='b')
            #float : conversion d'un caractère en nombre

            
            """ Elimination des cellules qui n'en sont pas. """
            # methode 0 
            mu = np.mean(list_intensite)
            std = np.std(list_intensite)
            density = gaussian_kde(list_intensite)
            density.covariance_factor = lambda:.25
            density._compute_covariance()
            xmin = min(list_intensite)
            xmax = max(list_intensite)
            x = np.linspace(xmin, xmax, len(list_intensite))
            plt.fill_between(x, 0, density(x), alpha=0.5)
            tmp=list()
            tmp.append(mu-3*std)
            tmp.append(mu+3*std)
            plt.plot(tmp,np.zeros_like(tmp), 'ro', clip_on=False)
            #plt.show()
            
            BF_Results_final=BF_Results
            BF_Results_final= BF_Results_final.loc[BF_Results_final["Mean"]>mu-3*std]
            BF_Results_final= BF_Results_final.loc[BF_Results_final["Mean"]<mu+3*std]
            
            #Recuperation du nombre de cellule en mitose
            result_champ_mitose = pd.read_csv(temperature+"/"+lignee+"/"+champ+"/BF_1_MITOSIS/Resultats_finaux.csv", sep=';')
            
            # Calcul du pourcentage de cellule en mitose
            nb_mitose = len(result_champ_mitose)
            nb_cellule = len(BF_Results_final)
            pourcentage_mitose = nb_mitose/nb_cellule*100
            #print (pourcentage_mitose)
            
            
            # Verification des laggings
            nb_lagging = 0
            if os.path.lexists(temperature+"/"+lignee+"/"+champ+"/BF_1_MITOSIS/abnormalCells.txt"):
                lagging_cell = pd.read_table(temperature+"/"+lignee+"/"+champ+"/BF_1_MITOSIS/abnormalCells.txt", header=None)
                for i in range (len(lagging_cell )) : 
                    if is_composed(str(lagging_cell.iloc[i]).split(':')[0], "Lagging") : 
                        nb_lagging = nb_lagging +1

            pourcentage_lagging = nb_lagging/nb_cellule*100
            
            """ Recuperation des durees et vitesses en metaphase et anaphase """
            #result_champ_mitose
            #vitesse metaphase
            res_vitesse_meta = result_champ_mitose["vitesse_meta"][result_champ_mitose["vitesse_meta"]>0]
            mean_vitesse_metaphase = np.mean (res_vitesse_meta)
            std_vitesse_metaphase = np.std (res_vitesse_meta)
            #vitesse anaphase
            res_vitesse_ana = result_champ_mitose["vitesse_ana"][result_champ_mitose["vitesse_ana"]>0]
            mean_vitesse_anaphase = np.mean (res_vitesse_ana)
            std_vitesse_anaphase = np.std (res_vitesse_ana)
            
            #duree metaphase
            res_duree_meta = result_champ_mitose["duree_meta"][result_champ_mitose["duree_meta"]!=0]
            mean_duree_metaphase = np.mean (res_duree_meta)
            std_duree_metaphase = np.std (res_duree_meta)
            #duree anaphase
            res_duree_ana = result_champ_mitose["duree_ana"][result_champ_mitose["duree_ana"]!=0]
            mean_duree_anaphase = np.mean (res_duree_ana)
            std_duree_anaphase = np.std (res_duree_ana)
            
            
            if lignee == "102":
                res_champ_102.loc[cpt_102] = ({"temperature":temperature,\
                                               "numero_champ":champ,\
                                               "nb_cellules":nb_cellule,\
                                               "nb_cellules_mitose":nb_mitose,\
                                               "%_mitose":pourcentage_mitose,\
                                               "nb_cellule_lagging":nb_lagging,\
                                               "%_lagging":pourcentage_lagging,\
                                               "moyenne_duree_metaphase":mean_duree_metaphase,\
                                               "std_duree_metaphase":std_duree_metaphase,\
                                               "moyenne_vitesse_metaphase":mean_vitesse_metaphase,\
                                               "std_vitesse_metaphase":std_vitesse_metaphase,\
                                               "moyenne_duree_anaphase":mean_duree_anaphase,\
                                               "std_duree_anaphase":std_duree_anaphase,
                                               "moyenne_vitesse_anaphase":mean_vitesse_anaphase,\
                                               "std_vitesse_anaphase":std_vitesse_anaphase})
                cpt_102=cpt_102+1
                
            elif lignee == "849":
                res_champ_849.loc[cpt_849] = ({"temperature":temperature,\
                                               "numero_champ":champ,\
                                               "nb_cellules":nb_cellule,\
                                               "nb_cellules_mitose":nb_mitose,\
                                               "%_mitose":pourcentage_mitose,\
                                               "nb_cellule_lagging":nb_lagging,\
                                               "%_lagging":pourcentage_lagging,\
                                               "moyenne_duree_metaphase":mean_duree_metaphase,\
                                               "std_duree_metaphase":std_duree_metaphase,\
                                               "moyenne_vitesse_metaphase":mean_vitesse_metaphase,\
                                               "std_vitesse_metaphase":std_vitesse_metaphase,\
                                               "moyenne_duree_anaphase":mean_duree_anaphase,\
                                               "std_duree_anaphase":std_duree_anaphase,
                                               "moyenne_vitesse_anaphase":mean_vitesse_anaphase,\
                                               "std_vitesse_anaphase":std_vitesse_anaphase})
                cpt_849=cpt_849+1
                
# sauvegarde du fichier
res_champ_102.to_csv("results_102.csv", sep=';')
res_champ_849.to_csv("results_849.csv", sep=';')

['102', '849']
['18C_102_1', '18C_102_2', '18C_102_3', '18C_102_4', '18C_102_5', '18C_102_6']
['18C_849_1', '18C_849_2', '18C_849_3', '18C_849_4', '18C_849_5', '18C_849_6']


# Creation du fichier de moyenne groupe par temperature